# Extracting Spectra

This notebook demonstrates how to grab 1D aperture summed HETDEX spectra for an input of ID, RA and DEC using the `Extract` Class API from `HETDEX_API`. This can be done most directly using the command line tool `get_spec.py`. It can be used both interactively on a compute node via `idev` or through a jupyter notebook as shown here, as well as within a slurm job file. If you find you need to run as a slurm job, please ask Erin to set you up on wrangler. We will not be using stampede2 for slurm jobs.

Examples of what you might like to do with the spectra afterwards is shown later. For example the spectra produced from get_spec.py can be translated to line intensities, or approximate pass-band magnitudes with a few extra steps.

#### Import all necessary python packages. 
These are mainly for working within the notebook. The command line tool already has the necessary preamble built in.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, join

from hetdex_tools.get_spec import get_spectra
from hetdex_api.survey import Survey
from hetdex_api.shot import *

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")


# IMPORTANT NOTE OF CAUTION WITH RUNNING ON TACC!!!

Because this script involves opening the Fibers class object which contains all fiber spectra from a 3 dither observation, you will be pulling in a lot of memory for each shot that is open. **NEVER** run this script from a login node on TACC. A login node is a node you access when you ssh in. 

You need to request a compute node instead by either 

(1) using the idev command :

`idev -p skx-dev `

if it works on a small catalog, you can always run interactively using a longer queue option

`idev -p skx-normal` 

(2) using a jupyter notebook 

(3) or by submitting the job into the slurm job scheduler (generally if you are working on more than ~200 shots

## Getting all spectra at a specified RA/DEC. This will search through all shots in HDR2

If a shotid is not specified the program will search for any shot within HDR2 that overlaps within an 11 arcmin radius of the input coordinates. Because of the non-contiguous VIRUS footprint, there is no guarantee the aperture defined by the input ra/dec/rad will contain enough fibers to do a measurement. The aperture radius is 3" by default or can be specified with the --rad argument. 

In [ ]:
from hetdex_tools.get_spec import get_spectra

Open a catalog of IDs, RAs, DECs:

In [ ]:
input_cat = Table.read('/work/05350/ecooper/stampede2/hdr2-tests/hps-muse/highz_emitters.fits')

In [ ]:
input_cat

`get_spectra()` requires an astropy coordinates object list as an input.

In [ ]:
input_coords = SkyCoord(ra=input_cat['RA'], dec=input_cat['DEC'])

In [ ]:
sources = get_spectra(input_coords, ID=input_cat['ID'])

# Reading in the output - astropy FITS files

In [ ]:
sources

Join your input and output table so you can match up any properties you like

In [ ]:
output_table = join(input_cat, sources)

In [ ]:
output_table

In [ ]:
sel = output_table['FLUX'] > 10

for row in output_table[sel][0:10]:
    plt.figure()
    wave_obj = row['WAVE']
    wave = row['wavelength']
    spec = row['spec']
    plt.plot(wave, spec)
    plt.xlim(wave_obj-50, wave_obj+50)
    plt.xlabel('wave')
    plt.ylabel('spec')
    plt.title(row['ID'])


# Examples of running get_spec as a command line job:


You can run these commands from the command line by removing the "!" command but be sure you are on a compute node by calling `idev` first.

In [ ]:
!hetdex_get_spec --ra 150.02548 --dec 2.087987 --ID cosmos_LAE --outfile cosmos_LAE

## Speed things up using multiprocessing

You can speed up processes (by up to ~30x) with python multiprocessing if you are working interactively in a notebook or in an idev session (**NEVER FROM A LOGIN NODE**). Use the multiprocessing option with the argument -mp True or --multiprocess True 

In [ ]:
!hetdex_get_spec --multiprocess -ra 150.02548 -dec 2.087987 -id mptest -o mptest

## Save output as individual astropy tables for each ID/shot combination:

If you prefer to save each spectra to a table, you can do this. We don't recommend this for large numbers of data, but understand that sometimes its easy to start with a readable table. The tables will be stored in the files named `spec_[ID]_[shotid].tab`

In [ ]:
!hetdex_get_spec  --multiprocess --single -ra 150.02548 -dec 2.087987 -id cosmos_lae

## Getting all spectra at a specified RA/DEC in a specific OBSERVATION/SHOT

Perhaps you only want to focus on a specific shot. Then you can use the -s argument to put the shotid either as an interger value 'YYYYMMDDOBS'= 20190104008 or as a str '20190104v009'. Note if you don't give an --ID option the default is 'DEX'

This is a command line routine so remove the "!" if you are running in a terminal.

In [ ]:
!hetdex_get_spec  -ra 8.86535 -dec 0.59352  -s 20190104008 -o 20190104008

This is particularly helpful if you plan to submit each shot as a separate task. For this reason, I suggest changing the default --outfile option to -o 20190104008 to create the output pickle file 20190104008

## Work on a list of ID/RA/DECs

This can either be a saved astropy table, or an space delimited text file with 3 columns where the columns are ID, RA, DEC. If you want more functionality with your input catalog, just talk to Erin. Note that running this job will take about 30 minutes so only execute if you want to wait around to explore the output.

In [ ]:
!cp /work/05350/ecooper/stampede2/3dhst/3dhst_input.cat .

In [ ]:
!hetdex_get_spec  --multiprocess -i '3dhst_input.cat' -o '3dhst'